In [1]:
import pandas as pd
import os

# === ETAPA 1: EXTRAÇÃO ===
def carregar_dados(caminho):
    tabelas = {
        os.path.splitext(arquivo)[0]: pd.read_csv(os.path.join(caminho, arquivo), encoding='latin1')
        for arquivo in os.listdir(caminho)
        if arquivo.endswith('.csv')
    }
    return tabelas

**Objetivo**:
Carregar automaticamente todos os arquivos .csv de um diretório específico, convertendo cada um em um DataFrame e armazenando-os em um dicionário nomeado conforme o arquivo.

**Detalhes técnicos**:
1. **os.listdir()** lista os arquivos do diretório.
2. **pd.read_csv()** lê cada CSV.
3. **encoding='latin1'** garante a leitura correta de acentos e caracteres especiais.

In [2]:
# === ETAPA 2: TRANSFORMAÇÃO ===

def corrigir_valores_numericos(df):
    """
    Corrige colunas numéricas que aparentam estar com casas decimais implícitas.
    A lógica detecta possíveis inflamentos de valores com base na média e mediana da coluna.
    """
    for col in df.select_dtypes(include='number').columns:
        mediana = df[col].median()
        media = df[col].mean()

        # Se a média estiver muito acima da mediana, pode haver inflação por falta de ponto decimal
        if media > mediana * 5:  
            print(f'[Corrigindo coluna]: {col} | Média: {media:.2f} | Mediana: {mediana:.2f}')
            df[col] = df[col] / 10  
    
    return df


def limpar_colunas(tabelas):
    for df in tabelas.values():
        df.columns = df.columns.str.strip()
    return tabelas

def transformar_dados(tabelas):
    orders = tabelas['orders']
    channels = tabelas['channels']
    deliveries = tabelas['deliveries']
    drivers = tabelas['drivers']
    hubs = tabelas['hubs']
    payments = tabelas['payments']
    stores = tabelas['stores']

    df = (
        orders
        .merge(channels, on='channel_id', how='left')
        .merge(stores, on='store_id', how='left')
        .merge(payments, left_on='order_id', right_on='payment_order_id', how='left')
        .merge(deliveries, on='delivery_order_id', how='left')
        .merge(drivers, on='driver_id', how='left')
        .merge(hubs, on='hub_id', how='left')
    )
    return df

def corrigir_valores_inflados(df, threshold=10):
    colunas_corrigidas = []
    
    for col in df.select_dtypes(include='number').columns:
        mediana = df[col].median()
        media = df[col].mean()

        if mediana > 0 and media / mediana > threshold:
            # Aplica correção iterativa até a relação entre média e mediana ficar aceitável
            while media / mediana > threshold:
                df[col] = df[col] / 10
                media = df[col].mean()
                mediana = df[col].median()
            
            colunas_corrigidas.append(col)

    if colunas_corrigidas:
        print(f"[INFO] Colunas corrigidas por inflamento de escala: {colunas_corrigidas}")
    
    return df

Essa etapa do processo de transformação de dados inclui a correção de valores numéricos inflacionados, a limpeza dos nomes das colunas e a junção de várias tabelas em uma única tabela consolidada. Essas transformações são fundamentais para garantir a consistência e a precisão dos dados antes de serem utilizados em análises ou visualizações.

In [3]:
# === ETAPA 2.1: TRATAMENTOS AVANÇADOS ===

def ajustar_tipos(df):
    colunas_data = [col for col in df.columns if 'date' in col.lower() or 'data' in col.lower()]
    for col in colunas_data:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].astype(str).str.strip().str.lower()
    
    return df

def tratar_nulos(df):
    for col in df.select_dtypes(include='number').columns:
        df[col].fillna(df[col].median(), inplace=True)

    for col in df.select_dtypes(include='object').columns:
        df[col].fillna('desconhecido', inplace=True)
    
    return df

def remover_duplicatas(df):
    return df.drop_duplicates()

def padronizar_texto(df):
    for col in df.select_dtypes(include='object').columns:
        df[col] = df[col].str.strip().str.lower().replace({'-': ' ', '_': ' '}, regex=True)
    return df

def remover_colunas_constantes(df):
    return df.loc[:, df.nunique() > 1]

def tratar_outliers_iqr(df):
    for col in df.select_dtypes(include='number').columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        df = df[(df[col] >= limite_inferior) & (df[col] <= limite_superior)]
    return df


A Etapa 2.1 foca em tratamentos avançados nos dados, abordando questões como tipos de dados, valores nulos, duplicatas, inconsistências de texto, colunas constantes e outliers. Essas transformações são cruciais para garantir que os dados sejam limpos, consistentes e adequados para análises ou modelagens avançadas.

In [4]:
# === ETAPA 3: CARGA ===
def salvar_dados(df, caminho_saida):
    df.to_csv(caminho_saida, index=False)

# === EXECUÇÃO DO ETL ===
if __name__ == '__main__':
    caminho_entrada = '../data/raw'
    caminho_saida = '../data/processed/dataset_final.csv'

    tabelas = carregar_dados(caminho_entrada)
    tabelas = limpar_colunas(tabelas)
    df_final = transformar_dados(tabelas)
    salvar_dados(df_final, caminho_saida)

    print("✅ ETL finalizado com sucesso!")

✅ ETL finalizado com sucesso!



Salva o DataFrame final processado em um novo arquivo .csv, pronto para uso posterior.